In [1]:
import time

from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import TensorBoard

from model import *
from data import *
import ImageTB

Using TensorFlow backend.


In [2]:
unet_m = unet_modify()
MODELS = [
{'model_name':'Unet_modify_500',
           'model' : unet_m,
           'train':'data/skyFinder',
           'val'  :'data/skyFinder',
         'icm' : 'rgb'},]

LOGS_PATH = './log'
if not os.path.exists(LOGS_PATH):
    os.makedirs(LOGS_PATH)
    
WEIGHTS_PATH = './models'
if not os.path.exists(WEIGHTS_PATH):
    os.makedirs(WEIGHTS_PATH)
    
BATCH_SIZE = 5
VAL_BATCH_SIZE = 5
EPOCH = 50

/home/dmitri/dev/BachelorDiploma/venv/lib/python3.8/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/home/dmitri/dev/BachelorDiploma/model.py:78: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


In [3]:
data_gen_args = dict(
    rotation_range=0.3,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

for model in MODELS:
#     lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)

    weights_name = model['model_name'] + '.{epoch:02d}.hdf5'
    weights_dir = os.path.join(WEIGHTS_PATH, weights_name)
    checkpoint = ModelCheckpoint(weights_dir, monitor='acc', verbose=1, save_best_only=True, mode='max')

    train_generator = BatchGenerator(
        batch_size=BATCH_SIZE,
        train_path=model['train'],
        image_folder='train',
        mask_folder='train_labels',
        aug_dict=data_gen_args,
        save_to_dir = None,
        image_color_mode = model['icm']
    )

    val_generator = BatchGenerator(
        batch_size=VAL_BATCH_SIZE,
        train_path=model['val'],
        image_folder='val',
        mask_folder='val_labels',
        aug_dict=data_gen_args,
        save_to_dir = None,
        image_color_mode = model['icm'])

    CURRENT_TIME = str(time.strftime("%Y%m%d-%H%M%S"))    
    log_name = "{}_{}".format(model['model_name'], CURRENT_TIME)
    log_dir = os.path.join(LOGS_PATH, log_name)
    os.makedirs(log_dir)
#     tbCallBack = ImageTB.ImagesTensorBoard(log_dir=log_dir, generator=val_generator)

    model = model['model']

    model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=EPOCH,
        callbacks=[

                checkpoint
            ],
        validation_data=val_generator,
        validation_steps=len(val_generator),

        max_queue_size=8,
        workers=4,
        use_multiprocessing=True)

Found 7587 images belonging to 1 classes.
Found 7587 images belonging to 1 classes.
Found 3495 images belonging to 1 classes.
Found 3495 images belonging to 1 classes.
Epoch 1/50
1517/1517 [==============================] - 933s 615ms/step - loss: 0.1920 - accuracy: 0.9347 - val_loss: 0.2618 - val_accuracy: 0.8848
Epoch 2/50


/home/dmitri/dev/BachelorDiploma/venv/lib/python3.8/site-packages/keras/callbacks/callbacks.py:706: RuntimeWarning: Can save best model only with acc available, skipping.
  warnings.warn('Can save best model only with %s available, '


 387/1517 [======>.......................] - ETA: 10:01 - loss: 0.1670 - accuracy: 0.9450